<a href="https://colab.research.google.com/github/aaditya3301/agents/blob/main/agenticai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- 1. Logic Core ---
DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def bfs_path(start, goal, H, W, grid):
    q = deque([start])
    parent = {start: None}
    visited = {start}
    while q:
        cur = q.popleft()
        if cur == goal:
            path = []
            while cur:
                path.append(cur)
                cur = parent[cur]
            return path[::-1]

        for dr, dc in DIRS:
            nr, nc = cur[0] + dr, cur[1] + dc
            if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] == 0 and (nr, nc) not in visited:
                visited.add((nr, nc))
                parent[(nr, nc)] = cur
                q.append((nr, nc))
    return None

class Agent:
    def __init__(self, id, start, color):
        self.id = id
        self.pos = start
        self.path = []
        self.color = color
        self.history = [start]
        self.task = None

    def step(self):
        if len(self.path) > 1:
            self.path.pop(0)
            self.pos = self.path[0]
            self.history.append(self.pos)
            return True # Moved
        return False # Didn't move

# --- 2. Setup World ---
H, W = 15, 15
grid = np.zeros((H, W), dtype=int)
random.seed(42)

# Reduced wall density slightly to prevent unreachable keys
for r in range(H):
    for c in range(W):
        if random.random() < 0.15: grid[r, c] = 1

starts = [(1, 1), (H-2, W-2)]
for s in starts: grid[s] = 0

keys = set()
while len(keys) < 8:
    p = (random.randrange(H), random.randrange(W))
    if grid[p] == 0 and p not in starts: keys.add(p)

agents = [
    Agent(0, starts[0], (0.0, 1.0, 1.0)), # Cyan
    Agent(1, starts[1], (1.0, 0.0, 1.0))  # Magenta
]

# --- 3. Run Simulation (With "Stuck" Detection) ---
frames = []
max_steps = 300
shared_keys = set(keys)
status_msg = "RUNNING"

for step in range(max_steps):
    # 1. Check Success
    if not shared_keys:
        status_msg = "MISSION COMPLETE"
        # Add one final frame to show the win
        frames.append({'agents': [a.pos for a in agents], 'trails': [list(a.history) for a in agents], 'keys': [], 'status': status_msg})
        break

    # 2. Assign tasks
    for a in agents:
        if not a.path and shared_keys:
            targets = list(shared_keys)
            target = min(targets, key=lambda k: abs(k[0]-a.pos[0]) + abs(k[1]-a.pos[1]))
            a.task = target
            path = bfs_path(a.pos, target, H, W, grid)
            if path: a.path = path

    # 3. Step agents
    moved_any = False
    for a in agents:
        did_move = a.step()
        if did_move: moved_any = True

        if a.pos in shared_keys:
            shared_keys.remove(a.pos)
            a.task = None
            a.path = []

    # 4. Save frame
    frames.append({
        'agents': [a.pos for a in agents],
        'trails': [list(a.history) for a in agents],
        'keys': list(shared_keys),
        'status': "RUNNING"
    })

    # 5. Stop if stuck (Keys exist but nobody moved and nobody has a path)
    if not moved_any and all(not a.path for a in agents):
        status_msg = "STUCK (Unreachable Key)"
        frames[-1]['status'] = status_msg
        break

# --- 4. Render Animation ---
print(f"Simulation finished. Total Steps: {len(frames)}. Opening window...")

fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#121212')

def render_frame(frame_idx):
    ax.clear()
    ax.axis('off')
    data = frames[frame_idx]

    # Draw Grid
    canvas = np.zeros((H, W, 3)) + 0.1
    canvas[grid == 1] = [0.05, 0.05, 0.1]
    ax.imshow(canvas, extent=[0, W, H, 0])

    # Draw Trails
    for i, trail in enumerate(data['trails']):
        if not trail: continue
        y, x = zip(*trail[-20:]) # Trail length
        color = agents[i].color
        ax.scatter(x, y, c=[color], s=30, alpha=0.3, marker='s')

    # Draw Keys
    if data['keys']:
        ky, kx = zip(*data['keys'])
        ax.scatter(kx, ky, c='#FFD700', s=120, marker='*', edgecolors='white', linewidth=0.5)

    # Draw Agents
    for i, pos in enumerate(data['agents']):
        ax.scatter(pos[1], pos[0], c=[agents[i].color], s=150, edgecolors='white', linewidth=1.5, zorder=10)

    # Dynamic Title
    status = data['status']
    color = 'white'
    if status == "MISSION COMPLETE": color = '#00FF00' # Green
    elif "STUCK" in status: color = '#FF0000' # Red

    ax.set_title(f"STEP: {frame_idx} | {status}", color=color, fontsize=12, fontweight='bold')
    ax.set_ylim(H-0.5, -0.5)
    ax.set_xlim(-0.5, W-0.5)

# repeat=False prevents it from restarting loop
anim = FuncAnimation(fig, render_frame, frames=len(frames), interval=100, repeat=False)

plt.show()

In [ ]:
import numpy as np
import heapq
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- A* Logic ---
def astar(start, goal, H, W, grid):
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])
    open_set = [(h(start, goal), 0, start, None)]
    came_from = {}
    g_score = {start: 0}

    while open_set:
        _, dist, cur, parent = heapq.heappop(open_set)
        if cur in came_from and g_score.get(cur, float('inf')) < dist: continue
        came_from[cur] = parent

        if cur == goal:
            path = []
            while cur:
                path.append(cur)
                cur = came_from[cur]
            return path[::-1]

        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr, nc] == 0:
                new_g = dist + 1
                if new_g < g_score.get((nr, nc), float('inf')):
                    g_score[(nr, nc)] = new_g
                    heapq.heappush(open_set, (new_g + h((nr, nc), goal), new_g, (nr, nc), cur))
    return None

# --- Setup ---
H, W = 12, 12
grid = np.zeros((H, W), dtype=int)
random.seed(7)
# Add random walls
for r in range(H):
    for c in range(W):
        if random.random() < 0.1: grid[r, c] = 1

# Create Dirt
dirty_cells = set()
for r in range(H):
    for c in range(W):
        if grid[r, c] == 0 and random.random() < 0.3:
            dirty_cells.add((r, c))

agents = [
    {'id': 0, 'pos': (0, 0), 'path': [], 'color': '#FF5722'}, # Deep Orange
    {'id': 1, 'pos': (H-1, W-1), 'path': [], 'color': '#00BCD4'} # Cyan
]

# --- Simulation ---
history_frames = []
max_steps = 400

for step in range(max_steps):
    if not dirty_cells: break

    for i, ag in enumerate(agents):
        # Logic: Agent 0 cleans left side, Agent 1 cleans right side (optimization)
        my_region = [(r,c) for r,c in dirty_cells if (c < W//2 if i==0 else c >= W//2)]
        # If my side is clean, help the other side
        targets = my_region if my_region else list(dirty_cells)

        if not ag['path'] and targets:
            target = min(targets, key=lambda x: abs(x[0]-ag['pos'][0]) + abs(x[1]-ag['pos'][1]))
            path = astar(ag['pos'], target, H, W, grid)
            if path: ag['path'] = path

        if ag['path']:
            if len(ag['path']) > 1:
                ag['path'].pop(0)
                ag['pos'] = ag['path'][0]
            elif len(ag['path']) == 1:
                 ag['pos'] = ag['path'][0] # Reached end

        if ag['pos'] in dirty_cells:
            dirty_cells.remove(ag['pos'])

    history_frames.append({'agents': [dict(a) for a in agents], 'dirt': list(dirty_cells)})

# --- Render ---
print(f"Cleaning finished. Frames: {len(history_frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#212121')

def update(frame_idx):
    ax.clear(); ax.axis('off')
    data = history_frames[frame_idx]

    floor = np.zeros((H, W, 3)) + 0.2
    floor[grid == 1] = [0.1, 0.1, 0.1]
    ax.imshow(floor, extent=[0, W, H, 0])

    if data['dirt']:
        dy, dx = zip(*data['dirt'])
        ax.scatter(dx, dy, c='#795548', s=120, marker='o', alpha=0.8, edgecolors='none')

    for ag in data['agents']:
        ax.scatter(ag['pos'][1], ag['pos'][0], c=ag['color'], s=220, edgecolors='white', linewidth=2)

    ax.set_title(f"CLEANING CREW | Dirt Left: {len(data['dirt'])}", fontsize=10, color='white')
    ax.set_xlim(-0.5, W-0.5); ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, update, frames=len(history_frames), interval=50, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- 1. Logic Core ---
def astar(start, goal, H, W, grid):
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])
    open_set = [(h(start, goal), 0, start, None)]
    came_from = {}
    g = {start: 0}
    while open_set:
        _, dist, cur, parent = heapq.heappop(open_set)
        if cur in came_from: continue
        came_from[cur] = parent
        if cur == goal:
            path = []
            while cur:
                path.append(cur)
                cur = came_from[cur] if cur in came_from else None
            return path[::-1]
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr, nc]==0:
                if dist+1 < g.get((nr, nc), float('inf')):
                    g[(nr, nc)] = dist+1
                    heapq.heappush(open_set, (dist+1+h((nr, nc), goal), dist+1, (nr, nc), cur))
    return [start]

# --- 2. Setup ---
H, W = 12, 12
grid = np.zeros((H, W), dtype=int)
random.seed(5)
for r in range(H):
    for c in range(W):
        if random.random() < 0.12: grid[r, c] = 1

start1, start2 = (1, 1), (1, W-2)
goal1, goal2 = (H-2, 1), (H-2, W-2)

# Plan
p1 = astar(start1, goal1, H, W, grid)
p2 = astar(start2, goal2, H, W, grid)
hist = {0: list(p1), 1: list(p2)}

# Resolve Collisions
t = 0
max_len = max(len(p1), len(p2)) + 20
while t < max_len:
    pos0 = hist[0][t] if t < len(hist[0]) else hist[0][-1]
    pos1 = hist[1][t] if t < len(hist[1]) else hist[1][-1]

    if pos0 == pos1 and t > 0: # Collision detected
        # Agent 1 waits
        hist[1].insert(t, hist[1][t-1])
        max_len += 1
    t += 1

# Normalize lengths
final_len = max(len(hist[0]), len(hist[1]))

# --- 3. Animation ---
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#121212')

def render(frame):
    ax.clear()
    ax.axis('off')

    # Grid
    canvas = np.zeros((H, W, 3)) + 0.1
    canvas[grid == 1] = [0.05, 0.05, 0.1]
    ax.imshow(canvas, extent=[0, W, H, 0])

    # Goals
    ax.scatter(goal1[1], goal1[0], c='#00FF00', marker='x', s=100, linewidth=3, label='Goal 1')
    ax.scatter(goal2[1], goal2[0], c='#FF00FF', marker='x', s=100, linewidth=3, label='Goal 2')

    # Agents
    positions = []
    for i, color in [(0, '#00FFFF'), (1, '#FF00FF')]:
        h = hist[i]
        pos = h[frame] if frame < len(h) else h[-1]
        positions.append(pos)

        # Trail
        if frame > 0:
            past = h[:frame+1][-10:]
            py, px = zip(*past)
            ax.plot(px, py, c=color, linewidth=2, alpha=0.5)

        ax.scatter(pos[1], pos[0], c=color, s=200, edgecolors='white')

    status = "MOVING"
    if frame >= final_len - 1: status = "ARRIVED"
    if positions[0] == positions[1]: status = "COLLISION (Error)" # Should not happen

    ax.set_title(f"PATH PLANNERS | Step: {frame} | {status}", color='white')
    ax.set_ylim(H-0.5, -0.5); ax.set_xlim(-0.5, W-0.5)

anim = FuncAnimation(fig, render, frames=final_len, interval=200, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- 1. Logic Core: A* Pathfinding ---
def astar(start, goal, H, W, grid):
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])
    # Priority Queue: (f_score, g_score, current_node, parent_node)
    open_set = [(h(start, goal), 0, start, None)]
    came_from = {}
    g_score = {start: 0}

    while open_set:
        _, dist, cur, parent = heapq.heappop(open_set)

        # Skip if we found a shorter way to this node already
        if cur in came_from and g_score.get(cur, float('inf')) < dist:
            continue

        came_from[cur] = parent

        if cur == goal:
            path = []
            while cur:
                path.append(cur)
                cur = came_from[cur]
            return path[::-1] # Return reversed path

        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] == 0:
                new_g = dist + 1
                if new_g < g_score.get((nr, nc), float('inf')):
                    g_score[(nr, nc)] = new_g
                    heapq.heappush(open_set, (new_g + h((nr, nc), goal), new_g, (nr, nc), cur))
    return None

# --- 2. Setup World ---
H, W = 14, 14
grid = np.zeros((H, W), dtype=int)
random.seed(11)
for r in range(H):
    for c in range(W):
        if random.random() < 0.05: grid[r, c] = 1

# Agents: [id, start_pos, color]
agents = [
    {'id': 0, 'pos': (0, 0), 'path': [], 'task': None, 'color': '#FFC107'}, # Amber
    {'id': 1, 'pos': (H-1, 0), 'path': [], 'task': None, 'color': '#03A9F4'}, # Light Blue
    {'id': 2, 'pos': (0, W-1), 'path': [], 'task': None, 'color': '#8BC34A'}  # Light Green
]

items = set((random.randint(1, H-2), random.randint(1, W-2)) for _ in range(8))

# --- 3. Simulation Loop ---
frames = []
max_steps = 300

for _ in range(max_steps):
    # Stop condition: No items left AND all agents have stopped moving
    if not items and all(not a['path'] for a in agents):
        break

    # 1. Assign Tasks
    for a in agents:
        if not a['task'] and items:
            # Find closest item
            target = min(items, key=lambda x: abs(x[0]-a['pos'][0]) + abs(x[1]-a['pos'][1]))
            a['task'] = target
            path = astar(a['pos'], target, H, W, grid)
            if path:
                a['path'] = path
            else:
                # If path fails (blocked), reset task
                a['task'] = None

    # 2. Move Agents
    for a in agents:
        if a['path']:
            if len(a['path']) > 1:
                a['path'].pop(0)       # Remove current position
                a['pos'] = a['path'][0] # Move to next

        # 3. Check Pickup
        if a['task'] and a['pos'] == a['task']:
            if a['pos'] in items:
                items.remove(a['pos'])
            a['task'] = None
            a['path'] = []

    frames.append({'agents': [a['pos'] for a in agents], 'items': list(items)})

# --- 4. Visualization ---
print(f"Simulation finished. Total frames: {len(frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#263238') # Industrial Dark Grey

def render(frame):
    ax.clear()
    ax.axis('off')
    data = frames[frame]

    # Draw Floor
    canvas = np.zeros((H, W, 3)) + 0.15
    canvas[grid == 1] = [0.05, 0.05, 0.05]
    ax.imshow(canvas, extent=[0, W, H, 0])

    # Draw Items (Crates)
    if data['items']:
        iy, ix = zip(*data['items'])
        ax.scatter(ix, iy, c='#FF5722', s=150, marker='s', edgecolors='black', linewidth=2, label='Crate')

    # Draw Agents
    for i, pos in enumerate(data['agents']):
        ax.scatter(pos[1], pos[0], c=agents[i]['color'], s=200, edgecolors='white', linewidth=2, zorder=10)
        ax.text(pos[1], pos[0], str(i), ha='center', va='center', color='black', fontweight='bold', fontsize=8)

    status = "WORKING" if data['items'] else "COMPLETE"
    ax.set_title(f"WAREHOUSE | Items Left: {len(data['items'])} | {status}", color='white')
    ax.set_xlim(-0.5, W-0.5)
    ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=100, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def bfs(start, goals, H, W, grid):
    q = deque([start]); parent = {start: None}; visited = {start}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = [];
            while cur: path.append(cur); cur = parent[cur]
            return path[::-1]
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0 and (nr,nc) not in visited:
                visited.add((nr,nc)); parent[(nr,nc)]=cur; q.append((nr,nc))
    return None

H, W = 15, 15
grid = np.zeros((H, W), dtype=int)
random.seed(3)
for r in range(H):
    for c in range(W):
        if random.random() < 0.1: grid[r, c] = 1

agents = [{'pos':s, 'path':[], 'hist':[s]} for s in [(0,0), (0,W-1), (H-1,0)]]
victims = set()
while len(victims) < 6:
    p = (random.randrange(H), random.randrange(W))
    if grid[p] == 0: victims.add(p)

frames = []
for _ in range(300):
    if not victims: break

    for a in agents:
        if not a['path'] and victims:
            path = bfs(a['pos'], victims, H, W, grid)
            if path: a['path'] = path

        if a['path']:
            if len(a['path']) > 1: a['path'].pop(0); a['pos'] = a['path'][0]
            a['hist'].append(a['pos'])

        if a['pos'] in victims: victims.remove(a['pos']); a['path'] = []

    frames.append({'agents': [a['pos'] for a in agents], 'victims': list(victims), 'trails': [list(a['hist']) for a in agents]})

fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('black')

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    canvas = np.zeros((H, W, 3)) + 0.1
    canvas[grid==1] = [0.3, 0.3, 0.3]
    ax.imshow(canvas, extent=[0, W, H, 0])

    # Victims
    if data['victims']:
        vy, vx = zip(*data['victims'])
        ax.scatter(vx, vy, c='red', marker='P', s=150, edgecolors='white')

    # Trails
    for t in data['trails']:
        if not t: continue
        ty, tx = zip(*t[-10:])
        ax.plot(tx, ty, c='#00E676', alpha=0.5, linewidth=2)

    # Agents
    for pos in data['agents']:
        ax.scatter(pos[1], pos[0], c='#00E676', s=180, edgecolors='black')

    ax.set_title(f"RESCUE SQUAD | Victims Left: {len(data['victims'])}", color='white')
    ax.set_ylim(H-0.5, -0.5); ax.set_xlim(-0.5, W-0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=100, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def astar(start, goal, H, W, grid):
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])
    open_set = [(h(start, goal), 0, start, None)]; came = {}; g = {start: 0}
    while open_set:
        _, dist, cur, parent = heapq.heappop(open_set)
        if cur == goal:
            path = []
            while cur: path.append(cur); cur = came.get(cur)
            return path[::-1]
        if cur in came and g.get(cur,float('inf'))<dist: continue
        came[cur] = parent
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0:
                if dist+1 < g.get((nr,nc),float('inf')):
                    g[(nr,nc)]=dist+1; heapq.heappush(open_set, (dist+1+h((nr,nc),goal), dist+1, (nr,nc), cur))
    return None

H, W = 16, 16
grid = np.zeros((H, W), dtype=int)
random.seed(99)
for r in range(H):
    for c in range(W):
        if random.random() < 0.04: grid[r, c] = 1

drones = [
    {'pos':(1,1), 'path':[], 'task':None, 'color':'#2979FF'},
    {'pos':(H-2,W-2), 'path':[], 'task':None, 'color':'#FF4081'}
]
packages = set((random.randint(1, H-2), random.randint(1, W-2)) for _ in range(6))

frames = []
for _ in range(400):
    if not packages and all(not d['task'] for d in drones): break

    for d in drones:
        if not d['task'] and packages:
            target = min(packages, key=lambda x: abs(x[0]-d['pos'][0])+abs(x[1]-d['pos'][1]))
            d['task'] = target
            packages.remove(target)
            path = astar(d['pos'], target, H, W, grid)
            if path: d['path'] = path

        if d['path']:
            if len(d['path']) > 1: d['path'].pop(0); d['pos'] = d['path'][0]

        if d['pos'] == d['task']: d['task'] = None

    # Visualization data
    active_tasks = [d['task'] for d in drones if d['task']]
    frames.append({'drones': [d['pos'] for d in drones], 'packs': list(packages) + active_tasks})

print(f"Delivery complete. Frames: {len(frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#0D47A1')

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    canvas = np.zeros((H, W, 3)) + 0.1
    canvas[grid==1] = [0, 0, 0.2]
    ax.imshow(canvas, extent=[0, W, H, 0], alpha=0.6)

    for x in range(W): ax.axvline(x-0.5, color='white', alpha=0.1)
    for y in range(H): ax.axhline(y-0.5, color='white', alpha=0.1)

    if data['packs']:
        py, px = zip(*[p for p in data['packs'] if p])
        ax.scatter(px, py, c='#FFD740', marker='D', s=100, edgecolors='black')

    for i, pos in enumerate(data['drones']):
        ax.scatter(pos[1], pos[0], c=drones[i]['color'], s=200, marker='o', edgecolors='white', linewidth=2)
        ax.scatter(pos[1], pos[0], c='white', s=50, marker='+')

    ax.set_title(f"DRONE DELIVERY | Step: {frame}", color='white')
    ax.set_xlim(-0.5, W-0.5); ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=100, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def bfs(start, goals, H, W, grid):
    q = deque([start]); parent = {start: None}; visited = {start}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            while cur: path.append(cur); cur = parent[cur]
            return path[::-1]
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0 and (nr,nc) not in visited:
                visited.add((nr,nc)); parent[(nr,nc)]=cur; q.append((nr,nc))
    return None

H, W = 12, 12
grid = np.zeros((H, W), dtype=int)
random.seed(8)
for r in range(H):
    for c in range(W):
        if random.random() < 0.05: grid[r, c] = 1

to_paint = {(r, c) for r in range(H) for c in range(W) if grid[r, c] == 0 and random.random() < 0.4}

painters = [
    {'pos':(0,0), 'path':[], 'color':'#E040FB', 'rem':{p for p in to_paint if (p[0]+p[1])%2==0}},
    {'pos':(H-1,W-1), 'path':[], 'color':'#00E5FF', 'rem':{p for p in to_paint if (p[0]+p[1])%2!=0}}
]

curr_painted = set()
frames = []

for _ in range(400):
    if all(not p['rem'] for p in painters): break

    for p in painters:
        if not p['path'] and p['rem']:
            target = min(p['rem'], key=lambda x: abs(x[0]-p['pos'][0])+abs(x[1]-p['pos'][1]))
            path = bfs(p['pos'], {target}, H, W, grid)
            if path: p['path'] = path

        if p['path']:
            if len(p['path'])>1: p['path'].pop(0); p['pos'] = p['path'][0]

        if p['pos'] in p['rem']:
            p['rem'].remove(p['pos'])
            curr_painted.add((p['pos'], p['color']))

    frames.append({'agents': [p['pos'] for p in painters], 'painted': list(curr_painted)})

print(f"Painting finished. Frames: {len(frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#212121')

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    canvas = np.zeros((H, W, 3)) + 0.2
    canvas[grid==1] = [0.1, 0.1, 0.1]
    ax.imshow(canvas, extent=[0, W, H, 0])

    for (pos, col) in data['painted']:
        ax.scatter(pos[1], pos[0], c=col, s=180, marker='s', alpha=0.9)

    for i, pos in enumerate(data['agents']):
        ax.scatter(pos[1], pos[0], c=painters[i]['color'], s=200, edgecolors='white', linewidth=2)

    ax.set_title(f"GRID PAINTERS | Painted: {len(data['painted'])}", color='white')
    ax.set_xlim(-0.5, W-0.5); ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=80, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def astar(start, goal, H, W, grid):
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])
    open_set = [(h(start, goal), 0, start, None)]; came = {}; g = {start: 0}
    while open_set:
        _, dist, cur, parent = heapq.heappop(open_set)
        if cur == goal:
            path = [];
            while cur: path.append(cur); cur = came.get(cur)
            return path[::-1]
        if cur in came and g.get(cur,float('inf'))<dist: continue
        came[cur] = parent
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0:
                if dist+1 < g.get((nr,nc),float('inf')):
                    g[(nr,nc)]=dist+1; heapq.heappush(open_set, (dist+1+h((nr,nc),goal), dist+1, (nr,nc), cur))
    return None

H, W = 14, 14
grid = np.zeros((H, W), dtype=int)
random.seed(13)
for r in range(H):
    for c in range(W):
        if random.random() < 0.07: grid[r, c] = 1

agents = [{'pos':p, 'path':[], 'task':None} for p in [(0,0), (H-1,W-1), (H-1,0)]]
resources = [(random.randint(1,H-2), random.randint(1,W-2)) for _ in range(12)]
queue = list(resources)

frames = []
max_steps = 500

for _ in range(max_steps):
    if not queue and all(not a['task'] for a in agents): break

    for a in agents:
        if not a['task'] and queue:
            a['task'] = queue.pop(0)
            path = astar(a['pos'], a['task'], H, W, grid)
            if path: a['path'] = path

        if a['path']:
            if len(a['path']) > 1: a['path'].pop(0); a['pos'] = a['path'][0]

        if a['pos'] == a['task']:
            a['task'] = None

    active_res = list(queue) + [a['task'] for a in agents if a['task']]
    frames.append({'agents': [a['pos'] for a in agents], 'res': active_res})

print(f"Simulation complete. Frames: {len(frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('#3E2723')

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    canvas = np.zeros((H, W, 3)) + 0.2
    canvas[grid==1] = [0.1, 0.05, 0.05]
    ax.imshow(canvas, extent=[0, W, H, 0])

    for r in data['res']:
        ax.scatter(r[1], r[0], c='#FFD700', marker='h', s=150, edgecolors='orange')

    for pos in data['agents']:
        ax.scatter(pos[1], pos[0], c='#CDDC39', s=180, edgecolors='black')

    ax.set_title(f"MINING BOTS | Remaining: {len(data['res'])}", color='white')
    ax.set_xlim(-0.5, W-0.5); ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=100, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def bfs(start, goals, H, W, grid):
    q = deque([start]); parent = {start: None}; visited = {start}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = [];
            while cur: path.append(cur); cur = parent[cur]
            return path[::-1]
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0 and (nr,nc) not in visited:
                visited.add((nr,nc)); parent[(nr,nc)]=cur; q.append((nr,nc))
    return None

H, W = 16, 16
grid = np.zeros((H, W), dtype=int)
random.seed(21)
for r in range(H):
    for c in range(W):
        if random.random() < 0.03: grid[r, c] = 1

agents = [{'pos':p, 'path':[], 'task':None} for p in [(0,0), (0,W-1), (H-1,W-1)]]
fires = {(random.randint(3,H-4), random.randint(3,W-4)) for _ in range(5)}

frames = []
for _ in range(400):
    if not fires: break

    # Assign
    for a in agents:
        if not a['task'] and fires:
            a['task'] = min(fires, key=lambda x: abs(x[0]-a['pos'][0]) + abs(x[1]-a['pos'][1]))
            path = bfs(a['pos'], {a['task']}, H, W, grid)
            if path: a['path'] = path

    # Move & Extinguish
    for a in agents:
        if a['path']:
            if len(a['path']) > 1: a['path'].pop(0); a['pos'] = a['path'][0]

        if a['pos'] in fires:
            fires.remove(a['pos'])
            a['task'] = None
            a['path'] = []

    # Spread
    new_fires = set(fires)
    for f in fires:
        for dr, dc in [(0,1),(1,0),(0,-1),(-1,0)]:
            nr, nc = f[0]+dr, f[1]+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0:
                if random.random() < 0.05: # 5% spread chance
                    new_fires.add((nr, nc))
    fires = new_fires

    frames.append({'agents': [a['pos'] for a in agents], 'fires': list(fires)})

fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('black')

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    canvas = np.zeros((H, W, 3)) + 0.1
    canvas[grid==1] = [0.3, 0.3, 0.3]
    ax.imshow(canvas, extent=[0, W, H, 0])

    if data['fires']:
        fy, fx = zip(*data['fires'])
        ax.scatter(fx, fy, c='#FF3D00', s=120, marker='^', alpha=0.8, label='Fire')

    for pos in data['agents']:
        ax.scatter(pos[1], pos[0], c='#2962FF', s=180, marker='o', edgecolors='white')

    ax.set_title(f"FIREFIGHTERS | Active Fires: {len(data['fires'])}", color='white')
    ax.set_ylim(H-0.5, -0.5); ax.set_xlim(-0.5, W-0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=100, repeat=False)
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- 1. Logic Core: BFS Pathfinding ---
def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    visited = {start}

    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            while cur:
                path.append(cur)
                cur = parent[cur]
            return path[::-1]

        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = cur[0]+dr, cur[1]+dc
            if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] == 0 and (nr, nc) not in visited:
                visited.add((nr, nc))
                parent[(nr, nc)] = cur
                q.append((nr, nc))
    return None

# --- 2. Setup World ---
H, W = 18, 18
real_grid = np.zeros((H, W), dtype=int)
random.seed(31)
# Add walls (invisible at first in a real game, but we use this for collision)
for r in range(H):
    for c in range(W):
        if random.random() < 0.06: real_grid[r, c] = 1

# Agents: Start at different corners/sides to maximize spread
agents = [
    {'id': 0, 'pos': (1, 1),   'path': [], 'color': '#FF1744'}, # Red
    {'id': 1, 'pos': (1, W-2), 'path': [], 'color': '#00E5FF'}, # Cyan
    {'id': 2, 'pos': (H-2, 1), 'path': [], 'color': '#76FF03'}  # Green
]

# Global set of all reachable floor cells
explored = set(a['pos'] for a in agents)
all_floor = {(r, c) for r in range(H) for c in range(W) if real_grid[r, c] == 0}
unexplored = all_floor - explored

# Partition map into 3 vertical strips to coordinate agents
cols_per = W // 3
regions = [set() for _ in range(3)]
for (r, c) in unexplored:
    # Assign cell to region 0, 1, or 2 based on column index
    idx = min(c // cols_per, 2)
    regions[idx].add((r, c))

# --- 3. Simulation ---
frames = []
max_steps = 600

for _ in range(max_steps):
    # Stop if all regions are empty (exploration done)
    if not any(regions): break

    for i, a in enumerate(agents):
        reg = regions[i]

        # Re-plan if idle and there are still unknown cells in my region
        if not a['path'] and reg:
            # Find nearest unexplored cell in my specific region
            target = min(reg, key=lambda x: abs(x[0]-a['pos'][0]) + abs(x[1]-a['pos'][1]))
            path = bfs(a['pos'], {target}, H, W, real_grid)
            if path:
                a['path'] = path
            else:
                # If unreachable, remove from region to prevent getting stuck
                reg.discard(target)

        # Move
        if a['path']:
            if len(a['path']) > 1:
                a['path'].pop(0)
                a['pos'] = a['path'][0]
            elif len(a['path']) == 1:
                a['pos'] = a['path'][0] # Reached target

        # Mark current position as explored (Reveal map)
        if a['pos'] in reg:
            reg.remove(a['pos'])
        explored.add(a['pos'])

    # Save state for animation
    frames.append({'agents': [a['pos'] for a in agents], 'explored': list(explored)})

# --- 4. Visualization (Fog of War Style) ---
print(f"Exploration complete. Frames: {len(frames)}")
fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor('black') # Outside border

def render(frame):
    ax.clear(); ax.axis('off')
    data = frames[frame]

    # Create visual map
    # 0 = Unexplored (Black)
    # 1 = Explored Floor (Grey)
    # Wall logic: We render walls only if they are near explored areas?
    # For simplicity, we render the 'Floor' lighting up.

    canvas = np.zeros((H, W, 3)) # Start Pitch Black

    # "Light up" explored areas
    for (r, c) in data['explored']:
        canvas[r, c] = [0.4, 0.4, 0.4] # Grey floor

    # Optional: Render walls faintly so we know where boundaries are
    canvas[real_grid == 1] = [0.1, 0.1, 0.1]

    ax.imshow(canvas, extent=[0, W, H, 0])

    # Draw Agents
    for i, pos in enumerate(data['agents']):
        color = agents[i]['color']
        # Agent dot
        ax.scatter(pos[1], pos[0], c=color, s=150, edgecolors='white', zorder=10)
        # "Scanner" ring effect
        ax.add_patch(plt.Circle((pos[1], pos[0]), 2, color=color, alpha=0.15))

    percent = int((len(data['explored']) / len(all_floor)) * 100)
    ax.set_title(f"MAP EXPLORATION | Coverage: {percent}%", color='white', fontweight='bold')
    ax.set_xlim(-0.5, W-0.5); ax.set_ylim(H-0.5, -0.5)

anim = FuncAnimation(fig, render, frames=len(frames), interval=50, repeat=False)
plt.show()